# 네이버 플러스 스토어 '탈모' 데이터 분석 노트북 (묶음수량/폰트 처리 포함)

입력 CSV 예상 컬럼:
`스토어, 제목, 원래가격, 현재가격, 할인율, 상품링크, 제조사, 브랜드, 모델명, 원산지, 두피타입, 모발타입, 타입, 제품형태, 용량, 세부제품특징, 향계열, 종류, 성분, 주요제품특징`

- 제목에 `2개/세트/x2/1+1` 등이 있으면 **묶음 수량**으로 인식하여 1개 기준 가격을 계산합니다.
- 그래프 한글 폰트 깨짐을 자동으로 방지합니다.

In [11]:
from pathlib import Path
import json, re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import datetime as dt

INPUT_CSV = "naver_hairloss_products_with_specs_20250929_PM114126.csv"   # 여기에 파일명 입력
OUTDIR = Path(f"out_{dt.datetime.now().strftime('%Y%m%d_%H%M%S')}")
CONFIG_JSON = None  # 예: 'config_keywords.json' (없으면 기본값 사용)

In [12]:
def setup_korean_font():
    from matplotlib import font_manager, rcParams
    candidates = ["AppleGothic","Malgun Gothic","NanumGothic","Noto Sans CJK KR","Noto Sans KR","NanumBarunGothic"]
    available = set(f.name for f in font_manager.fontManager.ttflist)
    chosen = None
    for c in candidates:
        if c in available:
            chosen = c; break
    if chosen:
        rcParams["font.family"] = chosen
    rcParams["axes.unicode_minus"] = False
    return chosen

setup_korean_font()

'Malgun Gothic'

In [13]:
DEFAULT_CONFIG = {
    "peptide_keywords": ["펩타이드", "peptide", "GHK", "GHK-Cu", "구리펩타이드", "copper peptide"],
    "low_stimulus_keywords": ["저자극", "무실리콘", "무향", "민감성", "무알콜", "약산성"],
    "clinical_keywords": ["임상", "피부과테스트", "더마테스트", "인체적용시험", "유효성"],
    "ingredient_keywords": [
        "미녹시딜","피나스테리드","카페인","비오틴","나이아신아마이드","덱스판테놀","징크피리치온",
        "세라마이드","징크","로즈마리","녹차","쑥","판테놀"
    ],
    "category_guess": {
        "샴푸": ["샴푸"],
        "토닉": ["토닉","스캘프 토닉","헤어토닉"],
        "앰플/세럼": ["앰플","세럼","에센스","부스터"],
        "오일/트리트먼트": ["오일","트리트먼트","팩","마스크"],
        "영양제(비의약품)": ["영양제","보충제","캡슐","정","환"]
    },
    "recent_days_for_new": 90
}

POSSIBLE_COLS = {
    "name": ["제목","상품명","제품명","title","name"],
    "store": ["스토어","스토어명","판매처","상호","store","seller"],
    "brand": ["브랜드","brand"],
    "category": ["카테고리","분류","종류","타입","제품형태","category"],
    "price": ["현재가격","할인가격","할인가","판매가격","가격","판매가","원래가격","정가","price"],
    "list_price": ["원래가격","정가"],
    "discount_price": ["현재가격","할인가격","할인가","판매가격","discount_price"],
    "discount_rate": ["할인율","할인율(%)","discount_rate"],
    "volume": ["용량","용량표기","중량","용량(g/ml)","용량(ml)","용량(g)","volume"],
    "date": ["등록일","갱신일","등록/갱신","date","updated_at","created_at"],
    "url": ["상품링크","링크","url"],
    "desc": ["성분","주요제품특징","세부제품특징","상세요약","상세","설명","요약","description","desc","썸네일ALT","썸네일텍스트"],
    "reviews": ["리뷰수","리뷰","구매평","reviews","review_count"],
    "rating": ["평점","rating","별점"]
}

In [14]:
def ensure_dir(p: Path): p.mkdir(parents=True, exist_ok=True); return p
def krw_fmt(x, pos=None):
    try: return f"{int(x):,}원"
    except: return str(x)
def load_config(path: str | None):
    if path and Path(path).exists():
        cfg = json.loads(Path(path).read_text(encoding='utf-8'))
        merged = DEFAULT_CONFIG.copy(); merged.update(cfg); return merged
    return DEFAULT_CONFIG
def guess_col(df, candidates):
    for c in candidates:
        if c in df.columns: return c
    lower_map = {c.lower(): c for c in df.columns}
    for c in candidates:
        if c.lower() in lower_map: return lower_map[c.lower()]
    return None
def build_schema(df):
    return {k: guess_col(df, v) for k,v in POSSIBLE_COLS.items()}

In [15]:
def to_numeric(s):
    if s is None: return None
    return (s.astype(str).str.replace(r"[^\d\.\-]","", regex=True)
            .replace({"": np.nan, "-": np.nan}).astype(float))
def parse_volume_to_ml(x: str):
    if pd.isna(x): return np.nan
    s = str(x).lower().replace(" ", "")
    m = re.findall(r"(\d+(?:\.\d+)?)\s*(?:x|\*)\s*(\d+(?:\.\d+)?)(ml|l|g|mg)", s)
    if m:
        qty = float(m[0][0]); val = float(m[0][1]); unit = m[0][2]
        per = val*(1000 if unit=="l" else 1) if unit in ["ml","l"] else val
        if unit=="mg": per = val/1000.0
        return qty * per
    m2 = re.findall(r"(\d+(?:\.\d+)?)(ml|l|g|mg)", s)
    if m2:
        val = float(m2[0][0]); unit = m2[0][1]
        if unit=="ml": return val
        if unit=="l": return val*1000.0
        if unit=="mg": return val/1000.0
        if unit=="g": return val
    return np.nan
def parse_pack_qty(name_text: str) -> int:
    if not isinstance(name_text, str):
        return 1
    s = name_text.lower()
    m = re.search(r'(\d+)\s*\+\s*(\d+)', s)
    if m:
        try: return max(1, int(m.group(1)) + int(m.group(2)))
        except: pass
    m = re.search(r'(?:x|\*)\s*(\d+)\b', s)
    if m:
        try: return max(1, int(m.group(1)))
        except: pass
    cand = 1
    for g in re.findall(r'(\d+)\s*(개|세트|set|입|ea|팩)\b', s):
        cand = max(cand, int(g[0]))
    for g in re.findall(r'(\d+)(개입|세트|입|ea|팩)', s):
        cand = max(cand, int(g[0]))
    return max(1, cand)
def clean_df(df, schema):
    for k in ["name","store","price"]:
        if not schema.get(k):
            raise ValueError(f"필수 컬럼 누락: {k}")
    out = pd.DataFrame()
    for k, col in schema.items():
        out[k] = df[col] if (col and col in df.columns) else np.nan
    out["price"] = to_numeric(out.get("price"))
    if schema.get("discount_price"): out["discount_price"] = to_numeric(out.get("discount_price"))
    if schema.get("list_price"): out["list_price"] = to_numeric(out.get("list_price"))
    if schema.get("discount_rate"): out["discount_rate"] = to_numeric(out.get("discount_rate"))
    if schema.get("rating"): out["rating"] = to_numeric(out.get("rating"))
    if schema.get("reviews"): out["reviews"] = to_numeric(out.get("reviews"))
    desc_cols = [schema.get("desc")] if schema.get("desc") else []
    for extra in ["성분","주요제품특징","세부제품특징"]:
        if extra in df.columns and extra not in desc_cols: desc_cols.append(extra)
    out["desc"] = df[desc_cols].astype(str).apply(lambda r: " ".join([x for x in r if x and x!="nan"]), axis=1) if desc_cols else ""
    vol_src = out.get("volume")
    if isinstance(vol_src, pd.Series) and vol_src.isna().all():
        vol_src = out["name"].fillna("").astype(str) + " " + out["desc"].fillna("").astype(str)
    out["volume_ml"] = vol_src.apply(parse_volume_to_ml) if isinstance(vol_src, pd.Series) else np.nan
    out["final_price"] = (out.get("discount_price").where(~pd.isna(out.get("discount_price")), out["price"]) if "discount_price" in out.columns else out["price"])
    if "list_price" in out.columns:
        out["list_price"] = out["list_price"].where(~pd.isna(out["list_price"]), out["final_price"]) 
    out["unit_count"] = out["name"].astype(str).apply(parse_pack_qty)
    out["final_price_per_unit"] = out["final_price"] / out["unit_count"]
    out["price_per_ml"] = out["final_price"]/out["volume_ml"]
    if schema.get("date"): out["date_parsed"] = pd.to_datetime(out["date"], errors="coerce")
    else: out["date_parsed"] = pd.NaT
    return out

In [16]:
def make_flags(df, cfg):
    text = (df["name"].fillna("").astype(str) + " " + df["desc"].fillna("").astype(str)).str.lower()
    def flag_any(keys):
        if not keys: return pd.Series([False]*len(df))
        pat = "|".join([re.escape(k.lower()) for k in keys])
        return text.str.contains(pat, regex=True)
    df["is_peptide"] = flag_any(cfg["peptide_keywords"])
    df["is_low_stimulus"] = flag_any(cfg["low_stimulus_keywords"])
    df["is_clinical"] = flag_any(cfg["clinical_keywords"])
    for k in cfg["ingredient_keywords"]:
        df[f"ING_{k}"] = text.str_contains = text.str.contains(re.escape(k.lower())).astype(int)
    if "category" not in df.columns or df["category"].isna().all():
        guess = np.array(["기타"]*len(df))
        for cname, keys in cfg["category_guess"].items():
            pat = "|".join([re.escape(k.lower()) for k in keys])
            guess = np.where(text.str.contains(pat), cname, guess)
        df["category"] = guess
    return df

In [17]:
def calc_kpis(df, cfg):
    price = df["final_price_per_unit"].dropna()
    k = {
        "n_products": len(df),
        "n_stores": df["store"].nunique(dropna=True),
        "n_brands": df["brand"].nunique(dropna=True) if "brand" in df.columns else None,
        "price_med": float(price.median()) if not price.empty else None,
        "price_q1": float(price.quantile(0.25)) if not price.empty else None,
        "price_q3": float(price.quantile(0.75)) if not price.empty else None,
        "recent_ratio": None,
        "hot_median": None,
        "spearman_price_reviews": None,
        "hhi_reviews": None
    }
    if "date_parsed" in df.columns and df["date_parsed"].notna().any():
        cutoff = pd.Timestamp(dt.datetime.now() - dt.timedelta(days=cfg["recent_days_for_new"]))
        k["recent_ratio"] = float((df["date_parsed"] >= cutoff).mean())
    return k

In [18]:
def save_price_hist(df, outdir):
    s = df["final_price_per_unit"].dropna()
    if len(s)==0: return None
    plt.figure(); plt.hist(s, bins=30)
    plt.xlabel("1개 기준 가격(원)"); plt.ylabel("상품 수")
    plt.gca().xaxis.set_major_formatter(FuncFormatter(krw_fmt))
    path = outdir/"chart_price_per_unit_hist.png"
    plt.tight_layout(); plt.savefig(path); plt.close(); return path
def save_category_bar(df, outdir):
    grp = df.groupby("category")["final_price_per_unit"].count().sort_values(ascending=False)
    if len(grp)==0: return None
    plt.figure(); plt.bar(grp.index.astype(str), grp.values)
    plt.xticks(rotation=45, ha="right"); plt.ylabel("상품 수")
    path = outdir/"chart_category_count_bar.png"
    plt.tight_layout(); plt.savefig(path); plt.close(); return path
def save_ingredient_bar(df, cfg, outdir):
    cols = [c for c in df.columns if c.startswith("ING_")]
    if not cols: return None
    sums = df[cols].sum().sort_values(ascending=False).head(15)
    plt.figure(); plt.barh(sums.index.str.replace("ING_","",regex=False), sums.values)
    plt.xlabel("키워드 등장 상품 수")
    path = outdir/"chart_ingredients_top15.png"
    plt.tight_layout(); plt.savefig(path); plt.close(); return path
def save_price_per_ml_box(df, outdir):
    d = df.dropna(subset=["price_per_ml","category"])
    if len(d)==0: return None
    cats = d["category"].astype(str).unique().tolist()
    data = [d.loc[d["category"].astype(str)==c, "price_per_ml"].values for c in cats]
    plt.figure(); plt.boxplot(data, labels=cats, showfliers=False)
    plt.ylabel("ml당 가격(원)")
    path = outdir/"chart_price_per_ml_box.png"
    plt.tight_layout(); plt.savefig(path); plt.close(); return path
def save_whitespace_matrix(df, outdir):
    pivot = pd.pivot_table(df, index="is_peptide", columns="is_low_stimulus", values="final_price_per_unit", aggfunc="mean")
    if pivot.size==0: return None
    plt.figure(); plt.imshow(pivot.values, aspect="auto")
    plt.xticks([0,1], ["저자극 False","저자극 True"] if pivot.shape[1]==2 else [str(c) for c in pivot.columns])
    plt.yticks([0,1], ["펩타이드 False","펩타이드 True"] if pivot.shape[0]==2 else [str(r) for r in pivot.index])
    plt.colorbar(label="평균 1개 가격(원)")
    path = outdir/"chart_whitespace_matrix.png"
    plt.tight_layout(); plt.savefig(path); plt.close(); return path

In [19]:
def write_markdown_report(outdir, kpis, chart_paths, df):
    def today_str(): return dt.datetime.now().strftime("%Y.%m.%d")
    lines = []
    lines.append(f"# 네이버 플러스 스토어 '탈모' 시장 스냅샷 ({today_str()})\n")
    lines.append("## 핵심 KPI")
    lines.append(f"- 총 상품 수: **{kpis.get('n_products',0):,}**개")
    lines.append(f"- 스토어 수: **{kpis.get('n_stores',0):,}**곳")
    if kpis.get("n_brands") is not None:
        lines.append(f"- 브랜드 수: **{kpis['n_brands']:,}**개")
    if kpis.get("recent_ratio") is not None:
        lines.append(f"- 최근 90일 신규/갱신 비중: **{kpis['recent_ratio']*100:.1f}%**")
    if kpis.get("price_med") is not None:
        lines.append(f"- **1개 기준 가격** 중앙값(IQR): **{int(kpis['price_med']):,}원** (Q1: {int(kpis['price_q1']):,}원, Q3: {int(kpis['price_q3']):,}원)")
    lines.append("- (참고) 본 시트에는 리뷰/평점 컬럼이 없어 수요/집중도 지표는 생략했습니다.")
    lines.append("\n## 인사이트(사업계획서 문장 템플릿)")
    pr_med = kpis.get("price_med")
    if pr_med is not None:
        lines.append(f"- 제목 내 묶음 수량 보정 후 1개 기준 가격 중앙값은 **{int(pr_med):,}원**.")
    lines.append("- 카테고리(종류/타입/제품형태 기반 추정)별 ml당 가격 비교 → 프리미엄/가성비 포지셔닝.")
    lines.append("- 성분/클레임 키워드 출현율로 트렌드 파악.")
    lines.append("- (화이트스페이스) 펩타이드×저자극 조합의 평균 1개 가격 차이 확인.")
    lines.append("\n## 차트")
    for p in chart_paths:
        if p is None: continue
        lines.append(f"![{p.name}]({p.name})")
    out_md = OUTDIR/"report.md"; out_md.write_text("\n".join(lines), encoding='utf-8'); out_md

In [20]:
CFG = load_config(CONFIG_JSON)
OUTDIR = ensure_dir(OUTDIR)
df_raw = pd.read_csv(INPUT_CSV, encoding='utf-8')
schema = build_schema(df_raw)
df = clean_df(df_raw, schema)
df = make_flags(df, CFG)
df.head(3)

,name,store,brand,category,price,list_price,discount_price,discount_rate,volume,date,...,ING_비오틴,ING_나이아신아마이드,ING_덱스판테놀,ING_징크피리치온,ING_세라마이드,ING_징크,ING_로즈마리,ING_녹차,ING_쑥,ING_판테놀
0,"바이브랩 4주 솔루션 초록 탈모 샴푸 우디플로럴머스크, 500ml, 1개",바이브랩,바이브랩,NaN,30900.0,32000.0,30900.0,0.03,500ml,NaN,...,0,0,0,0,0,0,0,0,0,0
1,"바이브랩 4주 솔루션 초록 탈모 샴푸 우디플로럴머스크, 500ml, 2개",바이브랩,바이브랩,NaN,60900.0,62000.0,60900.0,0.01,500ml,NaN,...,0,0,0,0,0,0,0,0,0,0
2,두피진정 탈모샴푸300ml+100ml 앰플증정,Solep,솔랩,NaN,25400.0,35000.0,25400.0,0.27,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


In [21]:
kpis = calc_kpis(df, CFG)
kpis

{'n_products': 10095,
 'n_stores': 3452,
 'n_brands': 1942,
 'price_med': 27000.0,
 'price_q1': 16980.0,
 'price_q3': 43732.5,
 'recent_ratio': None,
 'hot_median': None,
 'spearman_price_reviews': None,
 'hhi_reviews': None}

In [22]:
charts = []
charts.append(save_price_hist(df, OUTDIR))
charts.append(save_category_bar(df, OUTDIR))
charts.append(save_ingredient_bar(df, CFG, OUTDIR))
charts.append(save_price_per_ml_box(df, OUTDIR))
charts.append(save_whitespace_matrix(df, OUTDIR))
charts

C:\Users\kbjoo\AppData\Local\Temp\ipykernel_46412\1625927550.py:29: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  plt.figure(); plt.boxplot(data, labels=cats, showfliers=False)


[WindowsPath('out_20251023_173526/chart_price_per_unit_hist.png'),
 WindowsPath('out_20251023_173526/chart_category_count_bar.png'),
 WindowsPath('out_20251023_173526/chart_ingredients_top15.png'),
 WindowsPath('out_20251023_173526/chart_price_per_ml_box.png'),
 WindowsPath('out_20251023_173526/chart_whitespace_matrix.png')]

In [23]:
report_md = write_markdown_report(OUTDIR, kpis, charts, df)
report_md